In [0]:
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, Normalizer, MinMaxScaler,RobustScaler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.linalg import Vectors
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import StandardScaler
import time 
import h2o
from pysparkling import *
from pysparkling.ml import H2OXGBoost, H2ODeepLearning
from h2o.estimators import H2OXGBoostEstimator, H2ODeepLearningEstimator
from pyspark import SparkContext
from pyspark.sql import SparkSession
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import mlflow
import mlflow.sklearn

In [0]:
mlflow.set_experiment("/mlflow/Modeling")

In [0]:
df = spark.sql("select * from HouseSensor")
# df = df.drop("house")
# print(df.rdd.getNumPartitions())
df = df.coalesce(24)
df.cache()

Out[13]: DataFrame[house: string, lastSensorEventHours: int, lastSensorEventSeconds: int, lastSensorDayOfWeek: int, windowDuration: int, timeSinceLastSensorEvent: int, prevDominantSensor1: int, prevDominantSensor2: int, lastSensorID: int, lastSensorLocation: int, lastMotionLocation: int, complexity: double, activityChange: double, areaTransitions: int, numDistinctSensors: int, sensorCount_Bathroom: double, sensorCount_Bedroom: double, sensorCount_Chair: double, sensorCount_DiningRoom: double, sensorCount_Hall: double, sensorCount_Ignore: double, sensorCount_Kitchen: double, sensorCount_LivingRoom: double, sensorCount_Office: double, sensorCount_OutsideDoor: double, sensorCount_WorkArea: double, sensorElTime_Bathroom: double, sensorElTime_Bedroom: double, sensorElTime_Chair: double, sensorElTime_DiningRoom: double, sensorElTime_Hall: double, sensorElTime_Ignore: double, sensorElTime_Kitchen: double, sensorElTime_LivingRoom: double, sensorElTime_Office: double, sensorElTime_OutsideDoor: double, sensorElTime_WorkArea: double, activity: string]

In [0]:
house_stringIdx = StringIndexer(inputCol="house", outputCol="houses")
house = house_stringIdx.fit(df)
df = house.transform(df).drop('house')

In [0]:
label_stringIdx = StringIndexer(inputCol="activity", outputCol="label")
label = label_stringIdx.fit(df)
df_ML = label.transform(df).drop('activity')

In [0]:
feature_names = df_ML.columns
feature_names.remove('label')
va = VectorAssembler(outputCol="features", inputCols=feature_names)
va_df = va.transform(df_ML).select('features', 'label') 

In [0]:
[train_ML,test_ML] = va_df.randomSplit([0.8,0.2], seed=42)
train_ML.cache()
test_ML.cache()

Out[17]: DataFrame[features: vector, label: double]

In [0]:
LR_model = LogisticRegression(maxIter=100, family="multinomial")
DT_model = DecisionTreeClassifier(maxBins=100, maxDepth = 20)
RF_model = RandomForestClassifier(maxBins=100, maxDepth = 15, seed=42)

In [0]:
def modeling(model,train,test):
  start_time = time.time()
  trained_model = model.fit(train)
  preds = trained_model.transform(test)
  end_time = time.time()
  evaluator = MulticlassClassificationEvaluator()
  accuracy = evaluator.evaluate(preds)
  total_time = float(end_time-start_time)
  return  [accuracy*100, total_time, preds]

In [0]:
with mlflow.start_run():
  LR_model = LogisticRegression(maxIter=10)
  [acc_LR,time_taken, preds] = modeling(LR_model,train_ML,test_ML)
  print("Accuracy LR",acc_LR)
  print("time taken LR",time_taken)
#   mlflow.sklearn.log_model("Logistic Regression")
  mlflow.log_param("Model", "Logistic Regression")
#   mlflow.log_param("Max Iterations", 100)
  mlflow.log_metric("Time Taken", time_taken)
  mlflow.log_metric("Accuracy", acc_LR)

Accuracy LR 37.84783864600193
time taken LR 70.19494700431824

In [0]:
with mlflow.start_run():
  DT_model = DecisionTreeClassifier(maxBins=100, maxDepth=10)
  [acc_DT,time_taken,preds] = modeling(DT_model,train_ML,test_ML)
  print("Accuracy DT",acc_DT)
  print("time taken DT",time_taken)
#   mlflow.log_model("Decision treet")
  mlflow.log_param("Model", "Decision tree")
  mlflow.log_param("Max Bins", 100)
  mlflow.log_param("Max Depth", 20)
  mlflow.log_metric("Time Taken", time_taken)
  mlflow.log_metric("Accuracy", acc_DT)

Accuracy DT 52.648487067374305
time taken DT 20.644325494766235

In [0]:
with mlflow.start_run():
  RF_model = RandomForestClassifier(maxDepth=10, maxBins=100, numTrees=10, seed=42)
  [acc_RF,time_taken,preds] = modeling(RF_model,train_ML,test_ML)
  print("Accuracy RF",acc_RF)
  print("time taken RF",time_taken)
#   mlflow.log_model("Random Forest")
  mlflow.log_param("Model", "Random Forest")
  mlflow.log_param("Max Bins", 100)
  mlflow.log_param("Max Depth", 15)
  mlflow.log_metric("Time Taken", time_taken)
  mlflow.log_metric("Accuracy", acc_DT)

Accuracy RF 48.55541063909961
time taken RF 59.32688522338867

In [0]:
hc = H2OContext.getOrCreate()

Connecting to H2O server at http://10.139.64.7:9009 ... successful.
-------------------------- -------------------------------------------------------------------------------------------------------
H2O_cluster_uptime: 36 mins 24 secs
H2O_cluster_timezone: Etc/UTC
H2O_data_parsing_timezone: UTC
H2O_cluster_version: 3.32.0.2
H2O_cluster_version_age: 23 days
H2O_cluster_name: sparkling-water-root_app-20201211075837-0000
H2O_cluster_total_nodes: 3
H2O_cluster_free_memory: 38.71 Gb
H2O_cluster_total_cores: 24
H2O_cluster_allowed_cores: 24
H2O_cluster_status: locked, healthy
H2O_connection_url: http://10.139.64.7:9009
H2O_connection_proxy: null
H2O_internal_security: False
H2O_API_Extensions: XGBoost, Algos, Amazon S3, Sparkling Water REST API Extensions, AutoML, Core V3, TargetEncoder, Core V4
Python_version: 3.7.5 final
-------------------------- -------------------------------------------------------------------------------------------------------

Sparkling Water Context:
 * Sparkling Water Version: 3.32.0.2-1-3.0
 * H2O name: root
 * cluster size: 3
 * list of used nodes:
 (executorId, host, port)
 ------------------------
 (0,10.139.64.4,54321)
 (1,10.139.64.5,54321)
 (2,10.139.64.6,54321)
 ------------------------

 Open H2O Flow in browser: https://eastasia.azuredatabricks.net/driver-proxy/o/295627433871401/1104-121841-room558/9009/flow/index.html (CMD + click in Mac OSX)

In [0]:
sparkDF = hc.asSparkFrame(df)
sparkDF = df.withColumn("activity", df.activity.cast("string"))
[trainingDF, testingDF] = sparkDF.randomSplit([0.8, 0.2])
trainingDF.cache()
testingDF.cache()

Out[24]: DataFrame[lastSensorEventHours: int, lastSensorEventSeconds: int, lastSensorDayOfWeek: int, windowDuration: int, timeSinceLastSensorEvent: int, prevDominantSensor1: int, prevDominantSensor2: int, lastSensorID: int, lastSensorLocation: int, lastMotionLocation: int, complexity: double, activityChange: double, areaTransitions: int, numDistinctSensors: int, sensorCount_Bathroom: double, sensorCount_Bedroom: double, sensorCount_Chair: double, sensorCount_DiningRoom: double, sensorCount_Hall: double, sensorCount_Ignore: double, sensorCount_Kitchen: double, sensorCount_LivingRoom: double, sensorCount_Office: double, sensorCount_OutsideDoor: double, sensorCount_WorkArea: double, sensorElTime_Bathroom: double, sensorElTime_Bedroom: double, sensorElTime_Chair: double, sensorElTime_DiningRoom: double, sensorElTime_Hall: double, sensorElTime_Ignore: double, sensorElTime_Kitchen: double, sensorElTime_LivingRoom: double, sensorElTime_Office: double, sensorElTime_OutsideDoor: double, sensorElTime_WorkArea: double, activity: string, houses: double]

In [0]:
XGB_model = H2OXGBoost(labelCol = "activity")
DL_model = H2ODeepLearning(labelCol = "activity")

In [0]:
def modeling_H2O(model,train,test):
  start_time = time.time()
  model_H2O = model.fit(train)
  pred = model_H2O.transform(test)
  end_time = time.time()
  pred = pred.drop('detailed_prediction')
  df_pred = pred.toPandas()
  from sklearn.metrics import accuracy_score
  accuracy = accuracy_score(df_pred['activity'],df_pred['prediction'])
  total_time = float(end_time-start_time)
  return  [accuracy*100, total_time, df_pred]

In [0]:
with mlflow.start_run():
  XGB_model = H2OXGBoost(labelCol = "activity")
  [acc_XGB,time_taken, pred] = modeling_H2O(XGB_model,trainingDF, testingDF)
  print("Accuracy XGB",acc_XGB)
  print("time taken XGB",time_taken)
#   mlflow.log_model("XGBoost")
  mlflow.log_param("Model", "XGBoost")
  mlflow.log_metric("Time Taken", time_taken)
  mlflow.log_metric("Accuracy", acc_XGB)

Accuracy XGB 70.78281630518129
time taken XGB 1390.404586315155

In [0]:
with mlflow.start_run():
  DL_model = H2ODeepLearning(labelCol = "activity")
  [acc_DLE,time_taken, pred] = modeling_H2O(DL_model, trainingDF, testingDF)
  print("Accuracy DL",acc_DLE)
  print("time taken DL",time_taken)
#   mlflow.log_model("DNN")
  mlflow.log_param("Model", "DNN")
  mlflow.log_metric("Time Taken", time_taken)
  mlflow.log_metric("Accuracy", acc_DLE)

Accuracy DL 67.85354733118236
time taken DL 643.2962896823883